In [2]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

from bs4 import BeautifulSoup
import requests
import re

url = "https://www.iowa-demographics.com/zip_codes_by_population"
myRequest = requests.get(url)
soup = BeautifulSoup(myRequest.text,"html.parser")

zipPops = {}

# for code in soup.find_all('a'):
#     text = code.text
#     if text[0].isdigit():
#         print(text)

res = []

for code in soup.find_all('tr'):

    inner = code.find_all('td')

    if len(inner) == 0:
        continue
    else:
        for item in inner:
            item = str(item).split()
            res.append(item)
            

            # print(item)
        
res = res[:-1]
# print(res)

for i in range(0, len(res), 3):

    val = res[i][-3]

    #print(val)
    if val == "TIE":
        #print("do stuff")

        and_count = 0
        rel = res[i+1]

        to_add = []

        for j in range(len(rel)):


            if re.search(r'f=\"(\d+)', rel[j]) is not None:

                to_add.append(re.search(r'f=\"(\d+)', rel[j]).group(1))

        popspot = res[i+2][1]
        pop = re.search(r"\d+,?\d+", popspot).group()

        for zipcode in to_add:

            zipPops[int(zipcode)] = pop


    else:
        
    # if i % 3 == 2:
        codespot = res[i+1][2]
        code = re.search(r"\d+", codespot).group()
        popspot = res[i+2][1]
        pop = re.search(r"\d+,?\d+", popspot).group()
        zipPops[int(code)] = pop



#

# print(soup)


In [3]:
zipcode_data = pd.Series(zipPops).to_frame().reset_index()

In [4]:
zipcode_data = zipcode_data.rename(columns={"index": "Zip Code", 0: "Population"})

In [5]:
iowa = pd.read_csv("/Users/mdbarshay/Desktop/401/Project 1/iowa-sample.csv")

In [6]:
iowa["Date"] = iowa["Date"].astype("datetime64")

In [7]:
merged_iowa = pd.merge(iowa, zipcode_data, on = "Zip Code", how = "left")

In [8]:
merged_iowa["Population"] = merged_iowa["Population"].str.replace(",", "").astype(float)

In [16]:
final_simple = merged_iowa.groupby(["Zip Code", "Date", "Population"])[["State Bottle Cost", "State Bottle Retail", "Volume Sold (Liters)"]].sum()

In [17]:
final = final_simple.reset_index()

In [18]:
final

,Zip Code,Date,Population,State Bottle Cost,State Bottle Retail,Volume Sold (Liters)
0,50003,2013-08-27,7118.0,27.00,40.50,0.75
1,50009,2012-02-02,19559.0,21.21,31.81,0.80
2,50009,2012-07-12,19559.0,10.14,15.58,2.25
3,50009,2013-03-07,19559.0,9.97,14.96,9.00
4,50009,2013-12-30,19559.0,6.83,10.24,0.50
...,...,...,...,...,...,...
968,52807,2014-12-10,15638.0,10.49,15.74,9.00
969,52807,2015-02-04,15638.0,21.05,31.58,32.50
970,52807,2015-05-13,15638.0,26.49,39.74,1.75
971,52807,2015-08-12,15638.0,4.55,6.83,6.00


In [12]:
iowa

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S08096000008,2012-10-04,4641,Kum & Go #573 / SE 14th DM,5830 SE 14th ST,DES MOINES,50315,NaN,77,Polk,1022100,TEQUILA,395,Proximo,89191,Jose Cuervo Especial Reposado Tequila Mini,12,500,11.50,17.25,1,17.25,0.50,0.13
1,S23102300041,2014-12-20,4346,Roy's Foodland,105 PEARL ST,SHELLSBURG,52332,POINT (-91.869285 42.094155),6,Benton,1012100,CANADIAN WHISKIES,115,"Constellation Wine Company, Inc.",11774,Black Velvet,24,375,3.07,4.61,6,27.66,2.25,0.59
2,S14410500035,2013-09-09,3628,Wal-Mart 1528 / Cedar Rapids,2645 BLAIRS FERRY RD NE,CEDAR RAPIDS,52402,POINT (-91.680734 42.034748),57,Linn,1062300,FLAVORED RUM,260,Diageo Americas,43410,Captain Morgan Parrot Bay Coconut,12,750,7.49,11.23,12,134.76,9.00,2.38
3,S09427600021,2012-12-12,4708,No Frills Supermarkets #803 / Glenwo,423 SHARP ST,GLENWOOD,51534,POINT (-95.742987 41.04635),65,Mills,1011100,BLENDED WHISKIES,65,Jim Beam Brands,24458,Kessler Blend Whiskey,6,1750,11.01,16.52,30,495.60,52.50,13.87
4,S28446900114,2015-10-14,2594,Hy-Vee Food Store / Sioux City,4500 SERGEANT ROAD,SIOUX CITY,51106,POINT (-96.346969 42.447396),97,Woodbury,1012100,CANADIAN WHISKIES,115,"Constellation Wine Company, Inc.",10550,Black Velvet Toasted Caramel,12,750,6.75,10.13,12,121.56,9.00,2.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,S15723900021,2013-11-14,3997,Wal-Mart 4606 / Osceola,2400 COLLEGE DR,OSCEOLA,50213,POINT (-93.800214 41.024891000000004),20,Clarke,1011500,STRAIGHT RYE WHISKIES,255,Wilson Daniels Ltd.,27102,Templeton Rye,6,750,18.09,27.14,6,162.84,4.50,1.19
996,S05143400018,2012-04-19,4483,Sauce,108 E COLLEGE,IOWA CITY,52240,POINT (-91.533522 41.658979),52,Johnson,1081390,IMPORTED SCHNAPPS,421,"Sazerac Co., Inc.",69667,Dr. Mcgillicuddy's Vanilla,12,1000,11.00,16.50,12,198.00,12.00,3.17
997,S28018800016,2015-09-21,3926,Liquor Downtown / Iowa City,315 S GILBERT ST,IOWA CITY,52240,POINT (-91.530181 41.657654),52,Johnson,1011200,STRAIGHT BOURBON WHISKIES,65,Jim Beam Brands,27544,Red Stag By Jim Beam,12,750,10.49,15.74,12,188.88,9.00,2.38
998,S25060300052,2015-04-14,2353,Cresco Liquor Store,708 2ND AVE SE,CRESCO,52136,POINT (-92.106529 43.371131),45,Howard,1081240,GREEN CREME DE MENTHE,434,Luxco-St Louis,79336,Paramount Creme De Menthe/green,12,750,4.75,7.13,2,14.26,1.50,0.40
